# Little intro to SQLlite

In [52]:
import sqlite3
con = sqlite3.connect("tutorial.db")

In [55]:
cur.execute("CREATE TABLE movie(title, year, score)")

In [60]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

In [64]:
con.commit()

In [65]:
res = cur.execute("SELECT title FROM movie")
res.fetchall()

[('Monty Python and the Holy Grail',),
 ('And Now for Something Completely Different',),
 ('Monty Python and the Holy Grail',),
 ('And Now for Something Completely Different',),
 ('Monty Python and the Holy Grail',),
 ('And Now for Something Completely Different',)]

In [ ]:
cur = sqllite_connection.cursor()
cur.execute("DROP TABLE movie")

# Importing data and Loading to SQLlite

In [68]:
# Import necessary libraries and modules
from langchain.chat_models import ChatOpenAI
import sqlite3
import pandas as pd

# Set the OpenAI key
import mykey
OPENAI_API_KEY = mykey.OPENAI_API_KEY

# Initialize the Langchain ChatOpenAI 
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-1106")

#Reading my input data
retail_df = pd.read_csv("C:\\Users\\amanr\\OneDrive\\Desktop\\retail_sales_dataset.csv")

# Connect to the SQLite database
sqllite_connection = sqlite3.connect("amancustomer.db")

# Convert DataFrame to a SQLite table named "RetailSalesTable"
retail_df.to_sql("Sales", sqllite_connection, if_exists='replace')

1000

# Report generation via language models

In [78]:
# Import necessary libraries and modules
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase

# Create an instance of SQLDatabase using the 'customer.db' SQLite database
my_db = SQLDatabase.from_uri('sqlite:///amancustomer.db')

# Create an SQL agent executor with specified parameters
lc_agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=my_db, llm=llm),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
    return_intermediate_steps=True
)


C:\Users\amanr\anaconda3\lib\site-packages\langchain_community\agent_toolkits\sql\base.py:138: UserWarning: Received additional kwargs {'handle_parsing_errors': True, 'return_intermediate_steps': True} which are no longer supported.
  warnings.warn(


In [45]:
# Define user input - new one
user_inquiry = "What is the average sales across different product category?"

# Run the agent again
lc_agent_executor.run(user_inquiry)



> Entering new AgentExecutor chain...
I need to calculate the average sales for each product category in the database.
Action: sql_db_query_checker
Action Input: SELECT product_category, AVG(sales) FROM sales_table GROUP BY product_categorySELECT product_category, AVG(sales) 
FROM sales_table 
GROUP BY product_category;The query looks correct, but I should double-check with the database.
Action: sql_db_query
Action Input: SELECT product_category, AVG(sales) 
FROM sales_table 
GROUP BY product_category;Error: (sqlite3.OperationalError) no such table: sales_table
[SQL: SELECT product_category, AVG(sales) 
FROM sales_table 
GROUP BY product_category;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)I need to check for the correct table name and fields using sql_db_schema
Action: sql_db_list_tables
Action Input: SalesI should use sql_db_schema to query the correct table fields for the 'Sales' table.
Action: sql_db_schema
Action Input: Sales
CREATE TABLE "Sales" (
	"index" INT

'The average sales across different product categories are: Beauty - $467.48, Clothing - $443.25, Electronics - $458.79'

In [72]:
# Define user input - new one
user_inquiry = "What age group is buying most?"

# Run the agent again
lc_agent_executor.run(user_inquiry)



> Entering new AgentExecutor chain...
I should check the sales data to see which age group is making the most purchases.
Action: sql_db_query_checker
Action Input: SELECT age_group, COUNT(*) as total_purchases FROM sales_data GROUP BY age_group ORDER BY total_purchases DESC LIMIT 1;SELECT age_group, COUNT(*) as total_purchases 
FROM sales_data 
GROUP BY age_group 
ORDER BY total_purchases DESC 
LIMIT 1;I need to execute the query to get the result.
Action: sql_db_query
Action Input: SELECT age_group, COUNT(*) as total_purchases FROM sales_data GROUP BY age_group ORDER BY total_purchases DESC LIMIT 1;Error: (sqlite3.OperationalError) no such table: sales_data
[SQL: SELECT age_group, COUNT(*) as total_purchases FROM sales_data GROUP BY age_group ORDER BY total_purchases DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)I should check the list of tables in the database to see if sales_data exists.
Action: sql_db_list_tables
Action Input: SalesI need to check the

'64'

# The use of Prompt templates

In [49]:
from langchain_core.prompts import PromptTemplate

# Create the prompt template
template = PromptTemplate(
    input_variables=["user_inquiry", "background_info"],
    template="""{background_info}
    Question: {user_inquiry}
    """
)

# Define the background information
background_info = """
my role is to analyze the transaction patterns of customers. 
The feature engineering in table 'Sales' is crucial for statistical exploration. For example:
- column 'Age' can be grouped into bins of age ranges, such as 21-25, 26-30, and so on.
Understanding the data in these columns helps us gain insights about our customers, 
enabling us to offer personalized services and develop effective marketing strategies.
"""

# Define user input
user_inquiry = "What age group is buying most?"

# Run the agent with the formatted template
lc_agent_executor.run(template.format(background_info=background_info, user_inquiry=user_inquiry))



> Entering new AgentExecutor chain...
I need to examine the 'Sales' table to see the data in the 'Age' column.
Action: sql_db_schema
Action Input: Sales
CREATE TABLE "Sales" (
	"index" INTEGER, 
	"Transaction ID" INTEGER, 
	"Date" TEXT, 
	"Customer ID" TEXT, 
	"Gender" TEXT, 
	"Age" INTEGER, 
	"Product Category" TEXT, 
	"Quantity" INTEGER, 
	"Price per Unit" INTEGER, 
	"Total Amount" INTEGER
)

/*
3 rows from Sales table:
index	Transaction ID	Date	Customer ID	Gender	Age	Product Category	Quantity	Price per Unit	Total Amount
0	1	2023-11-24	CUST001	Male	34	Beauty	3	50	150
1	2	2023-02-27	CUST002	Female	26	Clothing	2	500	1000
2	3	2023-01-13	CUST003	Male	50	Electronics	1	30	30
*/I need to write a query to group the ages and count the number of transactions for each age group
Action: sql_db_query
Action Input: SELECT COUNT(*), 
    CASE 
        WHEN Age BETWEEN 21 AND 25 THEN '21-25' 
        WHEN Age BETWEEN 26 AND 30 THEN '26-30' 
        WHEN Age BETWEEN 31 AND 35 THEN '31-35' 
        

'Over 40'